This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

'wget' is not recognized as an internal or external command,
operable program or batch file.


1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [10]:
import pandas as pd
import numpy as np
data = pd.read_csv('data/data_000637.txt',nrows=15000)
data_full = pd.read_csv('data/data_000637.txt')
data.head()
df = pd.DataFrame(data)
df_full = pd.DataFrame(data_full)

2\. Estimate the number of BX in a ORBIT (the value 'x').

In [11]:
BX_max_value = df_full.BX_COUNTER.max()
print("Max BX number in a orbit: "+str(BX_max_value))

Max BX number in a orbit: 3563


3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [12]:
#Each TDC count corresponds to 25/30 ns,
#whereas a unit of BX_COUNTER corresponds to 25 ns, 
#and the ORBIT_CNT is increased every 'x' BX_COUNTER. 
#This allows to store the time in a similar way to hours, 
#minutes and seconds.
orbit_fraction = df_full.ORBIT_CNT.max()-df_full.ORBIT_CNT.min() #11004
bx_fraction = df_full.BX_COUNTER[len(df_full)-1] - df_full.BX_COUNTER[0] #-1612
tdc_fraction = df_full.TDC_MEAS[len(df_full)-1]-df_full.TDC_MEAS[0] #-8


total_time_ns = orbit_fraction*BX_max_value*25 + bx_fraction*25 + tdc_fraction
total_time_ns
td = pd.Timedelta(total_time_ns, unit='ns')
td

Timedelta('0 days 00:00:00.980140992')

In [13]:
import datetime as dt

itime = dt.datetime.now()
print("Begin time:", itime)

# the one-liner command
df['TIMENS'] = data['TDC_MEAS'] * 25 / 30 + data['BX_COUNTER'] * 25

ftime = dt.datetime.now()
print("End time:", ftime)
print("Elapsed time:", ftime - itime)
data

Begin time: 2021-12-03 04:15:00.828377
End time: 2021-12-03 04:15:00.828377
Elapsed time: 0:00:00


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIMENS
0,1,0,123,3869200167,2374,26,59371.666667
1,1,0,124,3869200167,2374,27,59372.500000
2,1,0,63,3869200167,2553,28,63848.333333
3,1,0,64,3869200167,2558,19,63965.833333
4,1,0,64,3869200167,2760,25,69020.833333
...,...,...,...,...,...,...,...
14995,1,1,4,3869200316,3399,9,84982.500000
14996,1,1,17,3869200316,3400,15,85012.500000
14997,1,1,10,3869200316,3530,16,88263.333333
14998,1,1,8,3869200316,3533,18,88340.000000


In [14]:
for i in range(1,len(df['TIMENS'])):
    df['TIMENS'][i]+=df['TIMENS'][i-1]

C:\Users\utente\AppData\Local\Temp/ipykernel_11508/1225524176.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TIMENS'][i]+=df['TIMENS'][i-1]


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIMENS,TOTAL_TIME(sec)
0,1,0,123,3869200167,2374,26,5.937167e+04,0.000059
1,1,0,124,3869200167,2374,27,1.187442e+05,0.000119
2,1,0,63,3869200167,2553,28,1.825925e+05,0.000183
3,1,0,64,3869200167,2558,19,2.465583e+05,0.000247
4,1,0,64,3869200167,2760,25,3.155792e+05,0.000316
...,...,...,...,...,...,...,...,...
14995,1,1,4,3869200316,3399,9,6.764781e+08,0.676478
14996,1,1,17,3869200316,3400,15,6.765631e+08,0.676563
14997,1,1,10,3869200316,3530,16,6.766513e+08,0.676651
14998,1,1,8,3869200316,3533,18,6.767397e+08,0.676740


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

In [35]:
# df_Time = pd.to_datetime((df_full['ORBIT_CNT']-df_full['ORBIT_CNT'][0])*BX_max_value*25 +(df_full['BX_COUNTER']-df_full['BX_COUNTER'][0])*25+df_full["TDC_MEAS"].astype(float)/1e9, unit='ns')
# df_time = (df_Time - pd.Timestamp("1970-01-01")) / pd.Timedelta('1ns')
# df_full['ABS_TIME(sec)'] = df_time/1e9
# df_full
# #df_full = df_full.drop(['ABS_TIME'],axis = 1)
s = pd.Series(pd.to_timedelta(df['TIMENS'], unit='ns'))
df['TOTAL_TIME(sec)'] = s.dt.total_seconds()

df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIMENS,TOTAL_TIME(sec)
0,1,0,123,3869200167,2374,26,5.937167e+04,0.000059
1,1,0,124,3869200167,2374,27,1.187442e+05,0.000119
2,1,0,63,3869200167,2553,28,1.825925e+05,0.000183
3,1,0,64,3869200167,2558,19,2.465583e+05,0.000247
4,1,0,64,3869200167,2760,25,3.155792e+05,0.000316
...,...,...,...,...,...,...,...,...
14995,1,1,4,3869200316,3399,9,6.764781e+08,0.676478
14996,1,1,17,3869200316,3400,15,6.765631e+08,0.676563
14997,1,1,10,3869200316,3530,16,6.766513e+08,0.676651
14998,1,1,8,3869200316,3533,18,6.767397e+08,0.676740


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [36]:
import numpy as np

df.groupby('TDC_CHANNEL').max()[-3:]

,HEAD,FPGA,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIMENS,TOTAL_TIME(sec)
TDC_CHANNEL,,,,,,,
127,1,1,3869200301,3337,29,6.053169e+08,0.605317
128,1,1,3869200306,3337,23,6.331016e+08,0.633102
139,1,1,3869200316,3560,0,6.768282e+08,0.676828


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [38]:
#len(df_full[df_full['TDC_CHANNEL'] == 139])
tdc_139 = df[(df['TDC_CHANNEL'])==139]
count_unique_139 = len(set(tdc_139.ORBIT_CNT))
tdc_139
print("Number of unique orbits with at least one measurement from TDC_CHANNEL=139: " + str(count_unique_139))
tdc_139

Number of unique orbits with at least one measurement from TDC_CHANNEL=139: 150


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIMENS,TOTAL_TIME(sec)
7,1,0,139,3869200167,2776,0,5.233442e+05,0.000523
15,1,1,139,3869200167,2797,0,1.080973e+06,0.001081
30,1,0,139,3869200167,3085,0,2.157544e+06,0.002158
39,1,0,139,3869200167,3186,0,2.865087e+06,0.002865
48,1,0,139,3869200168,23,0,3.105804e+06,0.003106
...,...,...,...,...,...,...,...,...
14956,1,0,139,3869200316,285,0,6.744718e+08,0.674472
14970,1,0,139,3869200316,1001,0,6.746896e+08,0.674690
14985,1,0,139,3869200316,3389,0,6.756302e+08,0.675630
14989,1,1,139,3869200316,3398,0,6.759691e+08,0.675969


7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel